In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [5]:
! pip install pretrainedmodels
! pip install wtfml

     |████████████████████████████████| 58 kB 1.7 MB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=e393092bb676edc82ea30461586617238710019c595e89f4217a64723cb2acbd
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [6]:
import torch
import torchvision
import albumentations
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm_notebook, tqdm
import pretrainedmodels
from wtfml.utils import EarlyStopping
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
print("Imported required packages. Using device: {}".format(device))

Imported required packages. Using device: cuda


In [7]:
warnings.simplefilter('ignore')
torch.manual_seed(42)
np.random.seed(42)

In [8]:
BASE_DIR = "../input/siim-isic-melanoma-classification/"

In [9]:
!ls -lrt "../input/siimisic-melanoma-resized-images"

total 10453644
-rw-r--r-- 1 nobody nogroup  101763200 May 30 19:55 x_train_32.npy
-rw-r--r-- 1 nobody nogroup   33736832 May 30 19:55 x_test_32.npy
-rw-r--r-- 1 nobody nogroup  134946944 May 30 19:55 x_test_64.npy
-rw-r--r-- 1 nobody nogroup  303630464 May 30 19:55 x_test_96.npy
-rw-r--r-- 1 nobody nogroup  407052416 May 30 19:55 x_train_64.npy
-rw-r--r-- 1 nobody nogroup  539787392 May 30 19:55 x_test_128.npy
-rw-r--r-- 1 nobody nogroup  915867776 May 30 19:56 x_train_96.npy
-rw-r--r-- 1 nobody nogroup 1628209280 May 30 19:56 x_train_128.npy
-rw-r--r-- 1 nobody nogroup 1653098624 May 30 19:56 x_test_224.npy
-rw-r--r-- 1 nobody nogroup 4986390656 May 30 19:56 x_train_224.npy


In [22]:
npy_data = np.load("../input/siimisic-melanoma-resized-images/x_train_128.npy")

# Dataloader Class

In [10]:
class MelanomaDataLoader(Dataset):
    '''Dataloader class'''
    def __init__(self, npy_data, targets, augmentations=None):
        self.npy_data = npy_data
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.npy_data)
    
    def __getitem__(self, idx):
        
        np_img = self.npy_data[idx]
        target = self.targets[idx]
        if self.augmentations:
            augmented = self.augmentations(image=np_img)
            image_data = augmented['image']
        else:
            image_data = torch.from_numpy(np_img)
        image_data = np.transpose(image_data, (2,0,1)).astype(np.float32)
        return {
            'images': torch.tensor(image_data, dtype=torch.float),
            'targets': torch.tensor(target, dtype=torch.long)
        }

# Model Class

In [11]:
class SEResnext50_32x4d(nn.Module):
    '''This is network class'''
    def __init__(self, pretrained='imagenet', wp = None):
        super(SEResnext50_32x4d, self).__init__()
        
        self.base_model = pretrainedmodels.__dict__['se_resnext50_32x4d'](pretrained=None)
        #print(self.base_model)
        if pretrained is not None:
            self.base_model.load_state_dict(
            torch.load('../input/pretrained-model-weights-pytorch/se_resnext50_32x4d-a260b3a4.pth')
            )
        '''for params in self.base_model.parameters():
            params.requires_grad = False'''
            
        self.l0 = nn.Linear(2048, 1)
        if wp is not None:
            self.criterion = nn.BCEWithLogitsLoss(pos_weight=wp)
        else:
            self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self, images, targets):
        batch_size = images.shape[0]
        
        x = self.base_model.features(images)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        yhat = self.l0(x)
        #loss = nn.BCEWithLogitsLoss(pos_weight=wp)(yhat, targets.view(-1, 1).type_as(x))
        loss = self.criterion(yhat, targets.view(-1, 1).type_as(x))
        return yhat, loss

In [9]:
!ls -lrt ../input/siim-isic-melanoma-classification/

total 4144
drwxr-xr-x 4 nobody nogroup    4096 May 27 22:18 jpeg
-rw-r--r-- 1 nobody nogroup  164748 May 27 22:19 sample_submission.csv
-rw-r--r-- 1 nobody nogroup  490920 May 27 22:19 test.csv
-rw-r--r-- 1 nobody nogroup 2056020 May 27 22:20 train.csv
drwxr-xr-x 2 nobody nogroup    4096 May 27 22:22 tfrecords
drwxr-xr-x 2 nobody nogroup  405504 May 27 22:22 test
drwxr-xr-x 2 nobody nogroup 1110016 May 27 22:23 train


In [12]:
# create folds
df = pd.read_csv(BASE_DIR+'train.csv')
df['fold'] = -1
#df = df.sample(frac=1).reset_index(drop=True)
y = df.target.values

kfolds = StratifiedKFold(n_splits=5)

for fold, (t_, v_) in enumerate(kfolds.split(X=df, y=y)):
    df.loc[v_, 'fold'] = fold
    
df.to_csv('./train_new.csv', index=False)

In [13]:
train_new = pd.read_csv('./train_new.csv')
print(train_new.head())
train_new.fold.value_counts()

     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   

  diagnosis benign_malignant  target  fold  
0   unknown           benign       0     0  
1   unknown           benign       0     0  
2     nevus           benign       0     0  
3   unknown           benign       0     0  
4   unknown           benign       0     0  


0    6626
4    6625
3    6625
2    6625
1    6625
Name: fold, dtype: int64

# Create Dataloaders

In [28]:
fold = 2
train_indices = train_new[train_new.fold != fold].index.to_numpy()
val_indices = train_new[train_new.fold == fold].index.to_numpy()
print(len(train_indices), len(val_indices))

26501 6625


In [29]:
print(train_indices[:5])

[0 1 2 3 4]


In [30]:
print(len(train_indices)+len(val_indices))
print(len(npy_data))

33126


NameError: name 'npy_data' is not defined

In [ ]:
print(set(train_indices).intersection(val_indices))

In [ ]:
fold=1
train_npy = npy_data[train_indices]
val_npy = npy_data[val_indices]
train_targets = train_new[train_new.fold != fold]['target'].to_numpy()
val_targets = train_new[train_new.fold == fold]['target'].to_numpy()
print(sum(train_targets==0))
print(sum(train_targets==1))
print(len(train_targets))

In [ ]:
#fold=0
train_unique, train_counts = np.unique(train_targets, return_counts=True)
val_unique, val_counts = np.unique(val_targets, return_counts=True)
print(f"Train counts: {train_unique} {train_counts} ********* Val counts: {val_unique} {val_counts}")

In [ ]:
#fold=1
train_unique, train_counts = np.unique(train_targets, return_counts=True)
val_unique, val_counts = np.unique(val_targets, return_counts=True)
print(f"Train counts: {train_unique} {train_counts} ********* Val counts: {val_unique} {val_counts}")

In [ ]:
print(f"There are {len(train_npy)} train data and {len(train_targets)} train targets. val count: {len(val_npy)}")

In [ ]:
#mean = (0.485, 0.456, 0.406)
#std = (0.229, 0.224, 0.225)
mean = (0.5,0.5,0.5)
std = (0.5,0.5,0.5)
train_aug = albumentations.Compose([
    albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
    albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
    albumentations.Flip(p=0.5)
])
valid_aug = albumentations.Compose([
    albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
])

In [ ]:
train_data = MelanomaDataLoader(train_npy, train_targets, augmentations=train_aug)
val_data = MelanomaDataLoader(val_npy, val_targets, augmentations=valid_aug)

In [ ]:
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4, shuffle=True)

In [ ]:
print(len(train_loader.dataset))

# Basic Visualization

In [ ]:
train_batch = next(iter(train_loader))
print(train_batch['images'].shape, train_batch['targets'])

In [ ]:
def imshow(img, title):
    plt.figure(figsize=(10,5))
    np_img = img.numpy() / 2 + 0.5
    plt.axis('off')
    plt.imshow(np.transpose(np_img, (1,2,0)))
    plt.title(title)
    plt.show()

In [ ]:
def show_image_batches(data_loader):
    batch = next(iter(data_loader))
    imgs, labels = batch['images'], batch['targets']
    print("img shape: ",batch['images'].shape)
    imgs = torchvision.utils.make_grid(imgs)
    title = labels.numpy().tolist()
    imshow(imgs, title)

In [ ]:
show_image_batches(train_loader)

In [ ]:
show_image_batches(val_loader)

# Dataloader time check

In [ ]:
BS = 16
train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BS, shuffle=True)

In [ ]:
%%time
batch = next(iter(train_loader))
print(batch['images'].shape)

In [14]:
def train(fold):
    epochs = 25
    BS = 64
    lr = 0.0001
    
    #Dataloader prep steps
    train_indices = train_new[train_new.fold != fold].index.to_numpy()
    val_indices = train_new[train_new.fold == fold].index.to_numpy()
    train_npy = npy_data[train_indices]
    val_npy = npy_data[val_indices]
    train_targets = train_new[train_new.fold != fold]['target'].to_numpy()
    val_targets = train_new[train_new.fold == fold]['target'].to_numpy()
    
    #let's check target distribution in this fold
    train_unique, train_counts = np.unique(train_targets, return_counts=True)
    val_unique, val_counts = np.unique(val_targets, return_counts=True)
    print(f"Train counts: {train_unique} {train_counts} ********* Val counts: {val_unique} {val_counts}")
    
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = albumentations.Compose([
        albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),
        albumentations.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
        albumentations.Flip(p=0.5)
    ])
    valid_aug = albumentations.Compose([
        albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
    ])
    train_data = MelanomaDataLoader(train_npy, train_targets, augmentations=train_aug)
    val_data = MelanomaDataLoader(val_npy, val_targets, augmentations=valid_aug)
    train_loader = DataLoader(train_data, batch_size=BS, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=BS, shuffle=False)
    
    wp = sum(train_targets==0) / sum(train_targets)
    fold_wp = torch.tensor(wp, dtype=torch.float)
    #modelling
    
    model = SEResnext50_32x4d(pretrained='imagenet', wp=fold_wp)
    
    model.to(device)
    
    '''for param in model.parameters():
        if param.requires_grad:
            print(param.shape)'''
    
    optimizr = torch.optim.Adam(model.parameters(), lr=lr)
    schedulr = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizr, patience=3, threshold=0.001, mode="max"
        )
    es = EarlyStopping(patience=5, mode='max')
    best_auc = 0
    losses = []
    n_iter = len(train_indices) // BS
    
    for epoch in range(epochs):
        model.train()
        
        tk0 = tqdm(train_loader, total=len(train_loader))
                    
        for i, data in enumerate(tk0, 1):
            images, targets = data['images'], data['targets']
            images, targets = images.to(device), targets.to(device)
            
            optimizr.zero_grad()
            #batch_wp = sum(targets==0) / sum(targets)
            out, loss = model(images, targets)
                           
            loss.backward()
            optimizr.step()
            
            optimizr.zero_grad()
            #train_unique, train_counts = np.unique(targets.cpu().numpy(), return_counts=True)
            #print(f"Train counts: {train_unique} {train_counts}")
            #print("Loss for batch: {} is {}".format(i+1, loss.item()))
            
            torch.cuda.empty_cache()
            
            #if i%50 == 0:
            #print(f"Batch {i} contains {sum(targets)} positive labels")
            #print("Evaluating model...")
            #print("Epoch: %d ******* Iter: %d/%d ******* Loss: %0.2f VAL_AUC: %0.2f"%(epoch, i, n_iter, loss.item(), val_auc))
            '''if val_auc > best_auc:
                print("Max AUC attained, saving model..")
                torch.save(model.state_dict(), './siimModel_{}.pth'.format(fold))
                best_auc = val_auc'''
            
            del images, targets
            
        
        val_auc = evaluate(val_loader, val_targets, model, device)
        print("Epoch: %d ******* VAL_AUC: %0.2f"%(epoch, val_auc))
        schedulr.step(val_auc)
        es(val_auc, model, model_path=f"./melanoma_fold_{fold}.bin")
        
        '''if val_auc > best_auc:
            print("Max AUC attained, saving model..")
            torch.save(model.state_dict(), './siimModel_{}.pth'.format(fold))
            best_auc = val_auc'''
            
        if es.early_stop:
            print("Early Stopping..")
            break

In [15]:
def evaluate(data_loader, val_targets, model, device):
    model = model.to(device)
    model.eval()
    final_preds = []
    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            images, targets = data['images'], data['targets']
            images, targets = images.to(device), targets.to(device)
            batch_wp = sum(targets==0) / sum(targets)
            
            preds, _ = model(images, targets)
            final_preds.append(preds.cpu())
    predictions = np.vstack((final_preds)).ravel()
    print('val_targets: ',val_targets[:5])
    print('predictions: ',predictions[:5])
    
    auc = roc_auc_score(val_targets, predictions)
    return auc

In [ ]:
train(0)
train(1)
train(2)
train(3)
train(4)

In [ ]:
!ls -lrt ../input/melanoma-pytorch-starter/

In [18]:
npy_test = np.load("../input/siimisic-melanoma-resized-images/x_test_64.npy")
print(f"There are {len(npy_test)} images in test set")
print(npy_test.shape)

There are 10982 images in test set
(10982, 64, 64, 3)


In [19]:
def predict(fold):
    BS = 4
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    test_aug = albumentations.Compose([
        albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
    ])
    #just for the sake
    test_targets = np.zeros(len(npy_test))
    test_wp = torch.tensor(1, dtype=torch.float)
    
    test_data = MelanomaDataLoader(npy_test, test_targets, augmentations=test_aug)
    test_loader = DataLoader(test_data, batch_size=BS, shuffle=False)
    
    model = SEResnext50_32x4d(pretrained=None, wp=test_wp)
    print(f"Loading from model: melanoma_fold_{fold}.bin")
    model.load_state_dict(torch.load(f"../input/melanoma-pytorch-starter/melanoma_fold_{fold}.bin"))
    model = model.to(device)
    model.eval()
    
    test_preds = []
    #tk1 = tqdm(test_loader, total = len(test_loader))
    with torch.no_grad():
        for batch, data in enumerate(test_loader, 1):
            torch.cuda.empty_cache()
            images, targets = data['images'], data['targets']
            images, targets = images.to(device), targets.to(device)
            out, _ = model(images, targets)
            #test_preds.append(out)
            test_preds.append(out.cpu())
            del images, targets
    predictions = np.vstack(test_preds).ravel()
    return predictions        

In [20]:
p1 = predict(1)

Loading from model: melanoma_fold_1.bin


In [23]:
print(type(p1))

<class 'numpy.ndarray'>


In [21]:
p0 = predict(0)
p2 = predict(2)
p3 = predict(3)
p4 = predict(4)

Loading from model: melanoma_fold_0.bin
Loading from model: melanoma_fold_2.bin
Loading from model: melanoma_fold_3.bin
Loading from model: melanoma_fold_4.bin


In [23]:
#submission
predictions = (p0 + p1 + p2 + p3 + p4) / 5
submission_df = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
submission_df.loc[:, 'target'] = predictions
print(submission_df.head())
submission_df.to_csv('submission_file.csv', index=False)

     image_name    target
0  ISIC_0052060 -6.285095
1  ISIC_0052349 -9.747205
2  ISIC_0058510 -7.569839
3  ISIC_0073313 -8.464066
4  ISIC_0073502 -2.361575


In [ ]:
#Check roc_auc
targets = np.zeros(10)
targets[8] = 1
print(targets)
preds = (np.random.rand(10)*0.1).ravel()
preds[8] = -0.1
print(preds)
auc = roc_auc_score(targets, preds)
print(auc)

In [ ]:
# check loss
out = np.array([-2.9445, -3.8510, -8.5114, 3.1692, 1.6949, -5.5680, -9.3456, -6.9603, -5.7006, -9.9718])
#out = (np.random.rand(10)*-10)
out = torch.from_numpy(out)
#out = out.view(-1,1)
print((out))
targets = torch.zeros(10)
targets[9] = 1
#targets = targets.view(-1,1)
print((targets))
print(targets.shape, out.shape)
wp = torch.tensor(9/1, dtype=torch.float)
loss = nn.BCEWithLogitsLoss(pos_weight=wp)(out, targets)
print(loss)

In [ ]:
# check loss
out = np.array([-2.9445, -3.8510, -8.5114, -3.1692, -1.6949, -5.5680, -9.3456, -6.9603, -5.7006, -5.9718])
#out = (np.random.rand(10)*-10)
out = torch.from_numpy(out)
#out = out.view(-1,1)
print((out))
targets = torch.zeros(10)
targets[9] = 1
#targets = targets.view(-1,1)
print((targets))
print(targets.shape, out.shape)
loss = nn.BCEWithLogitsLoss()(out, targets)
print(loss)